<a href="https://colab.research.google.com/github/joymaurya/QNA_BERT_Model/blob/main/BER_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets #install transformers and datasets modules

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.7 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset #importing load_dataset

In [3]:
db=load_dataset('squad') #Chossing SQuAD(Stanford Question and Answer Database) Dataset
db.filter(lambda x:len(x['answers']['text'])!=1)  # Removing Rows with multiple or zero answers for the same question

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Filter:   0%|          | 0/87599 [00:00<?, ? examples/s]

Filter:   0%|          | 0/10570 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 0
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10567
    })
})

In [4]:
from transformers import AutoTokenizer #importing autotokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased") #Importing Tokenizer for particular model distilBERT

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [5]:
#This function is created to return the starting and ending index of answers in context for training dataset
def answer_tokens(s_context_idx,e_context_idx,offset,s_ans_idx,e_ans_idx):
  index=s_context_idx                                 #starting index of context in the particular window
  start_index=0                                       #intialising Variable
  end_index=0                                         #intialising Variable
  if offset[s_context_idx][0]>s_ans_idx or offset[e_context_idx][1]<e_ans_idx:      #Using offset array which contains starting and ending index of all token in particular window to check if starting and ending index of answer is between the context of this window
      return 0,0                                      # return 0,0 if answer not found
  for k in offset[s_context_idx:]:                    #if answer is present btw the context , we find the starting index and ending index of answer in the context
    start,end=k
    if(start==s_ans_idx):                             #match the answer starting index with offset starting indexes
      start_index=index
    if(end==e_ans_idx):                               #match the answer ending index with offset ending indexes
      end_index=index
      break
    index=index+1
  return start_index,end_index                        #return starting index and ending index of answer with respect to context

In [6]:
#use this function to tokenize and get back list of starting indexes and ending indexes of answers
def tokenize_fn_train(batch):
  questions=[q.strip() for q in batch['question']]             #
  inputs=tokenizer(questions,batch['context'],
                   max_length=384,truncation='only_second',stride=128
                   ,return_overflowing_tokens=True,return_offsets_mapping=True,padding='max_length') #Tokenize the Data
  offset_mapping=inputs.pop('offset_mapping')                               # Removing offset_mapping which is not used as an input to model
  sample_idx=inputs.pop('overflow_to_sample_mapping')                       # Removing overflow_to_sample_mapping which is not used as an input to model
  answers=batch['answers']
  start_idx,end_idx=[],[]
  index=0
  for offsets,i in zip(offset_mapping,sample_idx):
    s_str_token=answers[i]['answer_start'][0]                                #starting index of answer in context (given in training data)
    e_str_token=s_str_token+len(answers[i]['text'][0])                       #finding end index of answer
    seq=inputs.sequence_ids(index)                                           #using sequence ids to find starting index of context
    s_context_index=seq.index(1)
    e_context_index=len(seq)-seq[::-1].index(1)-1
    tokens=answer_tokens(s_context_index,e_context_index,offsets,s_str_token,e_str_token)         # Using user defined function to get starting and ending index of single answer per loop
    start_idx.append(tokens[0])
    end_idx.append(tokens[0])
    index=index+1

  inputs['start_positions']=start_idx
  inputs['end_positions']=end_idx
  return inputs                                                              #returning inputs with addition of start_positions and end_positions

In [7]:
#mapping the training data through train tokenizer
train_dataset = db['train'].map(
     tokenize_fn_train,
     batched=True,
     remove_columns=db["train"].column_names,
 )

Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

In [8]:
# Tokenizing validation data
def tokenize_fn_validation(batch):
  questions=[q.strip() for q in batch['question']]
  inputs=tokenizer(questions,batch['context'],
                   max_length=384,truncation='only_second',stride=128
                   ,return_overflowing_tokens=True,return_offsets_mapping=True,padding='max_length') #Tokenize the Data
  sample_idx=inputs.pop('overflow_to_sample_mapping')                             #Removing overflow_to_sample_mapping
  sample_ids=[]
  for i in range(len(inputs['input_ids'])):
    sample=sample_idx[i]
    sample_ids.append(batch['id'][sample])                                        #appending unique id of each input to the array
    sequence_ids=inputs.sequence_ids(i)
    offset=inputs['offset_mapping'][i]
    inputs['offset_mapping'][i]=[
        x if sequence_ids[j] ==1 else None for j,x in enumerate(offset)           #changing values in offset_mapping which is not 1 to None (which means only )
    ]
  inputs['sample_id']=sample_ids
  return inputs                                                                   #returning inputs with addition of sample_id and change in offset_mapping

In [9]:
#mapping the training data through validation tokenizer
validation_dataset=db['validation'].map(
    tokenize_fn_validation,
    batched=True
    ,remove_columns=db['validation'].column_names)

Map:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [10]:
from transformers import TrainingArguments, Trainer    #importing modules

In [11]:
#setting TrainingArguments
training_args = TrainingArguments(
    'finetuned-squad3',
    evaluation_strategy="no",
    save_strategy='epoch',
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    fp16=True
)

In [12]:
#Using Distilbert model to fine-tune on over data
from transformers import AutoModelForQuestionAnswering
model = AutoModelForQuestionAnswering.from_pretrained("bert-base-cased")

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
#preparing Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    tokenizer=tokenizer,
)

In [14]:
trainer.train() #Training our Model

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,2.518800
1000,1.655300
1500,1.520700
2000,1.417300
2500,1.379300
3000,1.329600
3500,1.285800
4000,1.232000
4500,1.196200
5000,1.211900


TrainOutput(global_step=33276, training_loss=0.839889132012139, metrics={'train_runtime': 7919.7651, 'train_samples_per_second': 33.61, 'train_steps_per_second': 4.202, 'total_flos': 5.216534983896422e+16, 'train_loss': 0.839889132012139, 'epoch': 3.0})

In [63]:
vd=validation_dataset.select(range(250))

In [64]:
trainer_output=trainer.predict(vd) #Predicting Validation_dataset

In [65]:
predictions,_,_=trainer_output

In [66]:
start_logits,end_logits=predictions #prediction contains start and end logitS

In [105]:
#Function to predict the output based on start and end logits (ADDITION OF START AND END LOGIT VALUES GIVE ITS PROBAILITY OF OCCURANCE)
def predict_output(start_logits,end_logits,validation_dataset,raw_dataset):
  sample2idx={}
  for i , id_ in enumerate(validation_dataset['sample_id']):       #making dictionary mapping sample_ids to there respective index
    if id_ not in sample2idx:
      sample2idx[id_]=[i]
    else:
      sample2idx[id_].append(i)
  predicted_answers=[]
  for sample in raw_dataset:
    sample_id=sample['id']                                    #getting sample ID of input sample
    context=sample['context']                                        #saving sample context to var sample
    best_score=float('-inf')                                             #intiaizing best score as -infinity
    best_answer=None                                                               #intiatizing best answer as None
    for idx in sample2idx[sample_id]:#mapping back to index using sample_ids
      start_lits=start_logits[idx]
      end_lits=end_logits[idx]
      offsets=validation_dataset['offset_mapping'][idx]
      start_indices=(-start_lits).argsort()                  #sorting start_logits array in descending order
      end_indices=(-end_lits).argsort()                       #sorting end_logits array in descending order
      for start_idx in start_indices[:20]:                                   #using first 20 values for each start and end indices to predict the output
        for end_idx in end_indices[:20]:
          if offsets[end_idx]==None or offsets[start_idx]==None:
            continue
          if start_idx>end_idx:
            continue
          if end_idx-start_idx+1> 30:
            continue
          score=start_lits[start_idx]+end_lits[end_idx]
          if(score>best_score):                                         #storing the best_score
            best_score=score
            indx1=offsets[start_idx][0]                                 #getting starting index of best_answer in context using offset_mapping
            indx2=offsets[end_idx][1]                                   #getting end index of best_answer in context using offset_mapping
            best_answer=context[indx1:indx2]                            #storing best_answer
    predicted_answers.append({'id':sample_id,'prediction_text':best_answer})      #append in predicted_answers in proper format recognised by load_metric
  actual_answer=[]
  for i in raw_dataset:
    actual_answer.append({'id':i['id'],'answers':i['answers']})                      #append in predicted_answers in proper format recognised by load_metric
  from datasets import load_metric
  metric=load_metric('squad')
  return metric.compute(predictions=predicted_answers,references=actual_answer)     #Returning computational metric for predicted and actual answers

In [67]:
raw_data=db['validation'].select(range(250))

In [68]:
len(raw_data)

250

In [69]:
len(vd)

250

In [106]:
predict_output(start_logits,end_logits,vd,raw_data) #Compairing Output of 250 samples

/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for squad contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.0/metrics/squad/squad.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


{'exact_match': 40.0, 'f1': 61.32333333333336}

In [71]:
trainer.save_model('QNA_model_BERT') #Saving Model

In [72]:
from transformers import pipeline

In [73]:
qa=pipeline("question-answering",model='QNA_model_BERT',device=0) #importing over model through pipeline

In [100]:
#Giving own random Example to predict
context='Albert Einstein was a German-born theoretical physicist who is widely held to be one of the greatest and most influential scientists of all time .Einstein was ranked the greatest physicist of all time.He received the 1921 Nobel Prize in Physics "for his services to theoretical physics, and especially for his discovery of the law of the photoelectric effect a pivotal step in the development of quantum theory.'
question='In which year Albert Einstein won nobel prize?'
qa(context=context,question=question)

{'score': 0.9998465776443481, 'start': 217, 'end': 221, 'answer': '1921'}

In [95]:
def use__BERT():
  qa=pipeline("question-answering",model='QNA_model_BERT',device=0)
  context=input('context: ')
  question=input('question: ')
  k=qa(context=context,question=question)
  if k['score']<0.5 :
    print('Probabilty that this is the answer is low')
  print(k['answer'])

In [96]:
use__BERT()

context: Hi, this is Joy Maurya the writer of this code and i was born on 12th feb 2002
question: On which day write of this code was born
12th
